<a href="https://colab.research.google.com/github/prsdm/Large-Language-Models/blob/main/RAG_using_LangChain_and_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval-Augmented Generation (RAG) Notebook

Hello👋🏻 everyone!

I am Prasad and I am excited to share with you this notebook on Retrieval Augmented Generation (RAG).

In this notebook, you will learn how to implement RAG (basic to advanced) using LangChain 🦜 and LlamaIndex 🦙.

Feel free to explore, experiment, and connect with me on [LinkedIn](https://www.linkedin.com/in/prasad-mahamulkar/) and [Twitter](https://twitter.com/PrasadM1710) for any questions or discussions.

Happy coding! 🚀


# Basic RAG using LangChain

In [ ]:
! pip install sentence_transformers
! pip install pypdf
! pip install faiss-gpu
! pip install langchain
! pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.2 MB/s eta 0:00:00


In [ ]:
# load pdf
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/qlora_paper.pdf")
documents = loader.load()

In [ ]:
# split document content
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
text = text_splitter.split_documents(documents)

In [ ]:
# load embedding model
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5", encode_kwargs = {"normalize_embeddings": True})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/93.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# # check embeddings
# check = embeddings.embed_query("what is Qlora?")
# check[0:10]

In [ ]:
# create vectorstore using FAISS
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(text, embeddings)

In [ ]:
# saving the vectorstore
vectorstore.save_local("vectorstore.db")

In [ ]:
# # load the vectorstore and check similarity
# sub_docs = vectorstore.similarity_search("what is Qlora?") # k=5
# sub_docs

In [ ]:
# create retriever
retriever = vectorstore.as_retriever()

In [ ]:
# set api openai key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
# load llm
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [ ]:
# create document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

template = """"
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
doc_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
# create retrieval chain
from langchain.chains import create_retrieval_chain
chain = create_retrieval_chain(retriever, doc_chain)

In [ ]:
response = chain.invoke({"input": "what is Qlora?"})

In [ ]:
response['answer']

'QLoRA is an efficient finetuning approach that allows for the finetuning of quantized language models without any performance degradation. It reduces memory usage enough to finetune a 65B parameter model on a single 48GB GPU while preserving full 16-bit finetuning task performance.'

# Advanced RAG using LangChain

In [ ]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 4

In [ ]:
# load pdf
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/qlora_paper.pdf")
documents = loader.load()

In [ ]:
# split pages content
from langchain.text_splitter import RecursiveCharacterTextSplitter

# create the parent documents - The big chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# create the child documents - The small chunks
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# The storage layer for the parent chunks
from langchain.storage import InMemoryStore
store = InMemoryStore()

In [ ]:
# load embedding model
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5", encode_kwargs = {"normalize_embeddings": True})

In [ ]:
# create vectorstore using Chromadb
from langchain.vectorstores import Chroma
vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings)

In [ ]:
# create retriever
from langchain.retrievers import ParentDocumentRetriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
# add documents to vectorstore
retriever.add_documents(documents)

In [ ]:
# set api openai key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
# load llm
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [ ]:
# create document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

template = """"
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
doc_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
# create retrieval chain
from langchain.chains import create_retrieval_chain
chain = create_retrieval_chain(retriever, doc_chain)

In [ ]:
response = chain.invoke({"input": "what is Qlora?"})

In [ ]:
response['answer']

'QLORA is a method that achieves high-fidelity 4-bit finetuning by using two techniques: 4-bit NormalFloat (NF4) quantization and Double Quantization. It also introduces Paged Optimizers to prevent memory spikes during gradient checkpointing. QLORA has one low-precision storage data type (usually 4-bit) and one computation data type (usually BFloat16). It dequantizes the low-precision tensor to BFloat16 and performs matrix multiplication in 16-bit.'

# Basic RAG using LlamaIndex

In [ ]:
! pip install -U llama_hub llama_index pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
# load pdf
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_files=["/content/qlora_paper.pdf"]).load_data()

In [ ]:
# combine documents into one
from llama_index import Document
doc_text = "\n\n".join([d.get_content() for d in documents])
text= [Document(text=doc_text)]

In [ ]:
# set up text chunk
from llama_index.node_parser import SimpleNodeParser
node_parser = SimpleNodeParser.from_defaults() # Default chunk size is 1024

In [ ]:
# create chunks from text
base_nodes = node_parser.get_nodes_from_documents(text)

In [ ]:
# reset node ids
from llama_index.schema import IndexNode
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [ ]:
# load embedding model
from llama_index.embeddings import resolve_embed_model
embed_model = resolve_embed_model("local:BAAI/bge-small-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# set api openai key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
# load llm
from llama_index.llms import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
# set service context
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

In [ ]:
# store in vectorstore index
from llama_index import VectorStoreIndex
index = VectorStoreIndex(base_nodes, service_context=service_context)

In [ ]:
# create retriever
retriever = index.as_retriever() #similarity_top_k=2

In [ ]:
# set up query engine
from llama_index.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(retriever, service_context=service_context)

In [ ]:
# query
response = query_engine.query("What is Qlora?")
print(str(response))

QLORA is an efficient finetuning approach that allows for the reduction of memory usage while preserving the performance of a pretrained language model. It achieves this by backpropagating gradients through a frozen, 4-bit quantized pretrained language model into Low Rank Adapters (LoRA). QLORA introduces several innovations to save memory, including the use of a new data type called 4-bit NormalFloat (NF4), Double Quantization to reduce memory footprint, and Paged Optimizers to manage memory spikes. QLORA has been used to finetune more than 1,000 models and has shown state-of-the-art results in chatbot performance.


# Advanced RAG using LlamaIndex

In [ ]:
# load pdf
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_files=["/content/qlora_paper.pdf"]).load_data()

In [ ]:
# combine documents into one
from llama_index import Document
doc_text = "\n\n".join([d.get_content() for d in documents])
text= [Document(text=doc_text)]

In [ ]:
# set up parent chunk
from llama_index.node_parser import SimpleNodeParser
node_parser = SimpleNodeParser.from_defaults()

In [ ]:
from llama_index.schema import IndexNode
base_nodes = node_parser.get_nodes_from_documents(text)

In [ ]:
# reset node ids (optional)
from llama_index.schema import IndexNode
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [ ]:
# set up child chunk
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c,chunk_overlap=20) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [ ]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [ ]:
# set api openai key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [ ]:
# load llm
from llama_index.llms import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
# load embedding model
from llama_index.embeddings import resolve_embed_model
embed_model = resolve_embed_model("local:BAAI/bge-small-en")

In [ ]:
# set service context
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

In [ ]:
# create and store embedding in vectorstore
from llama_index import VectorStoreIndex
index = VectorStoreIndex(all_nodes, service_context=service_context)

In [ ]:
vector_retriever_chunk = index.as_retriever()

In [ ]:
# create retriever
from llama_index.retrievers import RecursiveRetriever
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [ ]:
# create retriever query
from llama_index.query_engine import RetrieverQueryEngine
query_engine_chunk = RetrieverQueryEngine.from_args(retriever_chunk, service_context=service_context)

In [ ]:
# # check source node
# from llama_index.response.notebook_utils import display_source_node
# nodes = retriever_chunk.retrieve( "what is Qlora?")
# for node in nodes:
#     display_source_node(node, source_length=2000)

In [ ]:
# query
response = query_engine_chunk.query("What is Qlora?")
print(str(response))

Retrieving with query id None: What is Qlora?
Retrieved node with id, entering: node-0
Retrieving with query id node-0: What is Qlora?
Retrieved node with id, entering: node-18
Retrieving with query id node-18: What is Qlora?
QLORA is an efficient finetuning approach that allows for the finetuning of quantized language models without any performance degradation. It reduces memory usage and enables the finetuning of large models on a single GPU. QLORA uses a novel technique to quantize a pretrained model to 4-bit and incorporates Low-rank Adapters (LoRA) for backpropagating gradients through the quantized weights. It introduces several innovations to save memory without sacrificing performance, such as a new data type called 4-bit NormalFloat (NF4), Double Quantization to reduce memory footprint, and Paged Optimizers to manage memory spikes. QLORA has been used to finetune more than 1,000 models and has achieved state-of-the-art results on various benchmarks.
